In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [03:07<00:00, 37.55s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Lowe's July 4th Grills & Outdoor Cooking Sale: Up to 55% off + free shipping w/ $35\nDetails: We've pictured the Pit Boss 850 DX Series 840-Sq. in Black Pellet Grill with Smart Compatibility for $499 ($50 off). Some of these deals require MyLowe's Rewards membership. (It's free to join.) Members also get free shipping on orders of $35 or more. Shop Now at Lowe's\nFeatures: \nURL: https://www.dealnews.com/Lowes-July-4-th-Grills-Outdoor-Cooking-Sale-Up-to-55-off-free-shipping-w-35/21749073.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Macy's July 4th Sale: Up to 80% off + free shipping w/ $25
Details: Shop the Macy's 4th of July sale and get deals up to 80% off on clothing, home goods, electronics, beauty, and more. We found great deals on brands like Levi's, Lacoste, Calvin Klein, adidas, and more. Plus, Star Rewards members will get free shipping on everything over $25 (it's free to join). Shop

In [9]:
gemini = os.getenv("GEMINI_API_KEY")
openai=OpenAI(api_key=gemini, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [12]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gemini-2.0-flash",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [13]:
result = get_recommendations()

In [14]:
len(result.deals)

5

In [15]:
result.deals[1]

Deal(product_description='The Sonos Era 100 is a powerful smart speaker that delivers high-quality audio. It features Alexa-enabled Bluetooth, Wi-Fi, and USB connectivity. With Trueplay tuning technology, the speaker analyzes the acoustics of your space to optimize the EQ. The Sonos app provides easy control and customization options. The model number is E10G1US1.', price=119.0, url='https://www.dealnews.com/products/Sonos/Sonos-Era-100-Bluetooth-Smart-Speaker/488007.html?iref=rss-c142')

In [16]:
from agents.scanner_agent import ScannerAgent

In [17]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result

DealSelection(deals=[Deal(product_description='This is a refurbished Pioneer DMH-W2770NEX multimedia receiver, designed for in-car entertainment and connectivity. It features a 6.8-inch capacitive touchscreen, offering intuitive control. The unit supports both Apple CarPlay and Android Auto for seamless smartphone integration, along with built-in Bluetooth and WiFi for wireless connectivity. Additionally, it is compatible with rear-view cameras and includes Amazon Alexa via the Vozsis app, enhancing convenience and safety on the road. A 1-year Allstate warranty is included with this device.', price=275.0, url='https://www.dealnews.com/products/Pioneer/Pioneer-DMH-W2770-NEX-Multimedia-Receiver-with-Wireless-Car-Play/490754.html?iref=rss-c238'), Deal(product_description="This comprehensive Jackery 5000 Plus 10kWh Whole Home Backup Kit is a solar generator system designed to provide uninterrupted power for your entire home. It features a Smart Transfer Switch for seamless power monitoring

: 